In [5]:
import pandas as pd
import numpy as np


In [6]:
df = pd.read_csv('top_men.csv')

In [7]:
df

,Name,Average Total,Apparatus
0,donnell whittenburg,14.592714,SR
1,asher hong,14.199900,SR
2,alex diab,14.158333,SR
3,stephen nedoroscik,14.726143,PH
4,brody malone,13.929800,PH
5,ian skirkey,13.783200,PH
6,khoi young,13.460000,PH
7,patrick hoopes,13.420000,PH
8,curran phillips,15.156250,PB
9,colt walker,15.019333,PB


In [18]:
# Create a Linear Programming problem
prob = pulp.LpProblem("Athlete_Selection", pulp.LpMaximize)

# Define the decision variables (binary variables for each athlete)
athletes = df['Name'].unique()
athlete_vars = pulp.LpVariable.dicts("Select", athletes, 0, 1, pulp.LpInteger)

# Define the objective function (maximize the sum of selected athletes)
objective = pulp.lpSum(athlete_vars[athlete] for athlete in athletes)
prob += objective

# Constraint: Select exactly 5 athletes
prob += pulp.lpSum(athlete_vars[athlete] for athlete in athletes) == 5

# Constraint: Each apparatus needs to be competed on 3 times
apparatus = df['Apparatus'].unique()
for event in apparatus:
    athletes_for_event = [athlete for athlete in athletes if df[(df['Name'] == athlete) & (df['Apparatus'] == event)].shape[0] > 0]
    prob += pulp.lpSum(athlete_vars[athlete] for athlete in athletes_for_event) == 3

# Solve the optimization problem using the CBC solver
prob.solve(pulp.PULP_CBC_CMD())

# Extract the selected athletes
selected_athletes = [athlete for athlete in athletes if pulp.value(athlete_vars[athlete]) == 1]

# Print the selected athletes and their respective apparatus
for athlete in selected_athletes:
    selected_apparatus = df[df['Name'] == athlete]['Apparatus'].tolist()
    print(f'{athlete} will compete on the following apparatus: {", ".join(selected_apparatus)}')


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/ryantalbot/opt/anaconda3/envs/tf2/lib/python3.9/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/j_/555m2zps099832fjh_m8jjnc0000gn/T/47ae13ca1fe84a2d9728415b4288a905-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/j_/555m2zps099832fjh_m8jjnc0000gn/T/47ae13ca1fe84a2d9728415b4288a905-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 10 COLUMNS
At line 46 RHS
At line 52 BOUNDS
At line 60 ENDATA
Problem MODEL has 5 rows, 7 columns and 14 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Problem is infeasible - 0.00 seconds
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.01

Khoi Young will compete on the following apparatus: PH
Fred Richard will compete on the following apparatus: PH
Paul Juda will compete on 